# Coffeepedia Data Cleaning Process

In [ ]:
# import dependencies
import pandas as pd
import json
from json import dumps

import warnings
warnings.filterwarnings("ignore")

First we load two CSV files--coffee commodity data from the USDA Foreign Agricultural Service (https://apps.fas.usda.gov/psdonline/app/index.html) and geographical coordinate data from Harvard University's WorldMap (https://worldmap.harvard.edu/data/geonode:country_centroids_az8).  We will then remove any extraneous columns, rename columns to create a common column for merging, and renaming any values to ensure all countries will be mapped to geocoordinates.

In [ ]:
# coffee commodity data
coffee = pd.read_csv("psd_coffee.csv")

# country latitude/longitude data
centroids = pd.read_csv("country_centroids_az8.csv")

In [ ]:
# reduce columns to only country name, latitude, longitude
latlong = centroids[["geounit", "Latitude", "Longitude"]]

# rename country name column to match column name on coffee df
latlong = latlong.rename(columns={"geounit":"Country_Name"})

In [ ]:
# rename countries to match all lat/long values
# coffee csv contains european country info for "European Union" as a group only
# so we must map the most geographically central country of the continent to the "EU" values
coffee["Country_Name"] = coffee["Country_Name"].replace({"Korea, South": "South Korea",
                                                         "Congo (Brazzaville)": "Republic of Congo",
                                                         "Congo (Kinshasa)": "Democratic Republic of the Congo",
                                                         "Cote d'Ivoire": "Ivory Coast",
                                                         "New Caledonia": "New Caledonia",
                                                         "Serbia": "Republic of Serbia",
                                                         "Yemen (Sanaa)": "Yemen",
                                                         "United States": "United States of America"
                                                        })
# now change "Czech Republic" back to "European Union"
latlong["Country_Name"] = latlong["Country_Name"].replace({"Czech Republic": "European Union"})

In [ ]:
# merge lat/long onto coffee df
coffee_latlong = pd.merge(coffee, latlong, on="Country_Name", how="left")

# remove extraneous columns
coffee_latlong_min = coffee_latlong[["Country_Name", "Market_Year", "Attribute_Description", "Value", "Latitude", "Longitude"]]

# export column-reduced df to csv as backup
coffee_latlong_min.to_csv("coffee_latlong_min.csv", index=False)

# Create JSON object to use in data.js file for data.html page

We then sent the coffee_latlong_min table to sqlite for storage. We queried the database and returned a JSON object to be used to populate the table on our data.html page. We limited this data to 2015-2019 to show multiple years, and converted to a JSON object. This will be pasted into the data.js file in the static/js folder and prepended with an appropriate variable to be used in the table building.

In [ ]:
# import more dependencies
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
engine = create_engine("sqlite:///coffee_data.sqlite")
conn = engine.connect()

# query db for 2019 arabica production, robusta production & domestic consumption

coffee_data = pd.read_sql("SELECT Country_Name,Market_Year, Value,Attribute_description, Latitude, Longitude from coffee_latlong_min WHERE Market_year in(2015,2016,2017,2018,2019)", conn)
coffee_data

coffee_datajson_file = json.loads(coffee_data.to_json(orient='records'))

print(coffee_datajson_file, file=open("data1.json", "a"))
print("Success!")

coffee_data1 = pd.read_sql("SELECT Country_Name,Market_Year, Value,Attribute_description, Latitude, Longitude from coffee_latlong_min WHERE Market_year in(2019)", conn)
print(coffee_data1)

# Create JSON objects to use in data.js file for Leaflet map

Next are creating JSON objects for the three attributes we will be mapping: arabica production, robusta production and domestic consumption. We are working with the df combining coffee data and latitude/longitude coordinates. Each JSON will be pasted into the data.js file in the static/js folder and prepended with an appropriate variable to be used in the map building.

In [ ]:
# limit data to 2019 only, remove extraneous columns. do this for each attribute
# arabica production
arabica_df = coffee_latlong_min.loc[(coffee_latlong_min["Attribute_Description"] == "Arabica Production") &
                                     (coffee_latlong_min["Market_Year"] == 2019)]
arabica_df = arabica_df[["Country_Name", "Value", "Latitude", "Longitude"]]

# robusta production
robusta_df = coffee_latlong_min.loc[(coffee_latlong_min["Attribute_Description"] == "Robusta Production") &
                                     (coffee_latlong_min["Market_Year"] == 2019)]
robusta_df = robusta_df[["Country_Name", "Value", "Latitude", "Longitude"]]

# domestic consumption
consump_df = coffee_latlong_min.loc[(coffee_latlong_min["Attribute_Description"] == "Domestic Consumption") &
                                     (coffee_latlong_min["Market_Year"] == 2019)]
consump_df = consump_df[["Country_Name", "Value", "Latitude", "Longitude"]]

# convert to JSON
arabica_json = json.loads(arabica_df.to_json(orient='records'))
robusta_json = json.loads(robusta_df.to_json(orient='records'))
consump_json = json.loads(consump_df.to_json(orient='records'))

# arabica_json
# robusta_json
# consump_json

# Create a table of all attributes and countries for a single year, 2019

Here we create a table of all attributes and countries for a single year, 2019, same as the map year. First we limit the data to 2019 and remove extraneous columns. Then we replace the spaces (" ") between words with underscores ("_") because these values will become keys when we access them in the Javascript files. We then pivot the table to assign the country names as index with each attribute as a column. We then export the cleaned csv to the data folder of our app.

In [ ]:
# limit data to 2019 only, for all countries, all attributes
coffee_2019 = coffee_latlong_min.loc[coffee_latlong_min["Market_Year"] == 2019]
coffee_2019 = coffee_2019[["Country_Name", "Attribute_Description", "Value"]]

# replace spaces between words in Attribute_Description column because they will become column names
coffee_2019 ["Attribute_Description"] = coffee_2019["Attribute_Description"].str.replace(' ', '_', regex=True)

# pivot the df to set country as index with columns for each attribute
pivot_2019 = coffee_2019.pivot(index="Country_Name", columns="Attribute_Description", values="Value")
pivot_2019

# send to csv in the app's data folder
pivot_2019.to_csv("../static/data/data_2019.csv")

#  Clean the data for HighCharts Streamgraph

This data will be used to create a streamgraph charting production trends over time for all countries. The data is sorted to display only the desired attributes and then pivoted to align country names as the index and years as the columns, illustrating change over time.

In [ ]:
# query arabica production for all countries and years
arabica = coffee_latlong_min.loc[coffee_latlong_min["Attribute_Description"] == "Arabica Production"]
arabica = arabica[["Country_Name", "Market_Year", "Attribute_Description", "Value"]]
arabica_pivot = arabica.pivot(index="Market_Year", columns="Country_Name")["Value"]

# query robusta production for all countries and years
robusta = coffee_latlong_min.loc[coffee_latlong_min["Attribute_Description"] == "Robusta Production"]
robusta = robusta[["Country_Name", "Market_Year", "Attribute_Description", "Value"]]
robusta_pivot = robusta.pivot(index="Market_Year", columns="Country_Name")["Value"]

# query domestic consumption for all countries and years
consump = coffee_latlong_min.loc[coffee_latlong_min["Attribute_Description"] == "Domestic Consumption"]
consump = consump[["Country_Name", "Market_Year", "Attribute_Description", "Value"]]
consump_pivot = consump.pivot(index="Market_Year", columns="Country_Name")["Value"]

# fill NaNs with 0 to retain as much data as possible
arabica_pivot = arabica_pivot.fillna(0)
robusta_pivot = robusta_pivot.fillna(0)
consump_pivot = consump_pivot.fillna(0)

# send to csv in the data folder of the app
arabica_pivot.to_csv("../static/data/arabica_pivot.csv")
robusta_pivot.to_csv("../static/data/robusta_pivot.csv")
consump_pivot.to_csv("../static/data/consump_pivot.csv")